## Part 1: Advanced Baseline Analysis (45 min)

### Exercise 1.1: Deep Dive into Revenue Components

In [ ]:
# Setup
import sys
sys.path.insert(0, '../../..')

from core.simulation import Simulation
from core.data_loader import DataLoader
from core.economics import EconomicEngine
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

print("✅ PoliSim modules loaded")

In [ ]:
# Exercise 1.1a: Load and examine revenue breakdown
loader = DataLoader()
baseline = loader.load_cbo_baseline()

# Revenue components for analysis
revenue_components = {
    'Individual Income Tax': 2176,  # billions, FY2024
    'Payroll Taxes': 1694,
    'Corporate Income Tax': 420,
    'Excise Taxes': 76,
    'Other': 154
}

# Calculate percentages
total = sum(revenue_components.values())
for source, amount in revenue_components.items():
    pct = (amount / total) * 100
    print(f"{source}: ${amount}B ({pct:.1f}%)")

In [ ]:
# Exercise 1.1b: Visualize revenue trends
years = list(range(2024, 2035))

# Projected revenue growth by source (nominal)
individual_tax = [2176 * (1.04)**i for i in range(11)]  # 4% growth
payroll_tax = [1694 * (1.035)**i for i in range(11)]    # 3.5% growth
corporate_tax = [420 * (1.03)**i for i in range(11)]    # 3% growth

plt.figure(figsize=(12, 6))
plt.stackplot(years, individual_tax, payroll_tax, corporate_tax,
              labels=['Individual Income Tax', 'Payroll Taxes', 'Corporate Tax'],
              colors=['#1f77b4', '#ff7f0e', '#2ca02c'], alpha=0.8)
plt.title('Projected Federal Revenue by Source (2024-2034)')
plt.xlabel('Fiscal Year')
plt.ylabel('Revenue ($ Billions)')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Exercise 1.2: Spending Decomposition

Understand what's driving spending growth.

In [ ]:
# Exercise 1.2a: Mandatory vs Discretionary spending
spending_2024 = {
    'Social Security': 1461,
    'Medicare': 874,
    'Medicaid': 616,
    'Other Mandatory': 818,
    'Defense': 886,
    'Non-Defense Discretionary': 744,
    'Net Interest': 870
}

# Growth rates (annual, nominal)
growth_rates = {
    'Social Security': 0.055,  # 5.5% - aging population + COLA
    'Medicare': 0.065,         # 6.5% - healthcare inflation + aging
    'Medicaid': 0.045,         # 4.5% - healthcare costs
    'Other Mandatory': 0.03,   # 3%
    'Defense': 0.025,          # 2.5% - constrained
    'Non-Defense Discretionary': 0.02,  # 2%
    'Net Interest': 0.08       # 8% - rising rates + debt
}

# Project spending to 2034
spending_2034 = {}
for category, base in spending_2024.items():
    rate = growth_rates[category]
    spending_2034[category] = base * (1 + rate)**10

print("Spending Growth 2024 → 2034:")
print("="*50)
for cat in spending_2024:
    growth = (spending_2034[cat] / spending_2024[cat] - 1) * 100
    print(f"{cat}: ${spending_2024[cat]:,}B → ${spending_2034[cat]:,.0f}B (+{growth:.0f}%)")

In [ ]:
# Exercise 1.2b: Visualize spending composition change
categories = list(spending_2024.keys())
values_2024 = list(spending_2024.values())
values_2034 = [spending_2034[cat] for cat in categories]

x = np.arange(len(categories))
width = 0.35

fig, ax = plt.subplots(figsize=(14, 6))
bars1 = ax.bar(x - width/2, values_2024, width, label='FY2024', color='steelblue')
bars2 = ax.bar(x + width/2, values_2034, width, label='FY2034 (Projected)', color='coral')

ax.set_ylabel('Spending ($ Billions)')
ax.set_title('Federal Spending by Category: 2024 vs 2034')
ax.set_xticks(x)
ax.set_xticklabels(categories, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### 📝 Analysis Questions - Part 1

1. Which spending category shows the fastest growth rate?
2. What happens to the mandatory/discretionary split by 2034?
3. Why is net interest growing so fast?
4. How does revenue growth compare to spending growth?

---

## Part 2: Healthcare Policy Comparison (60 min)

### Exercise 2.1: Compare Healthcare Systems

In [ ]:
# Exercise 2.1a: Load healthcare module
try:
    from core.healthcare import HealthcareEngine
    from core.context_aware_healthcare import ContextAwareHealthcare
    print("✅ Healthcare modules loaded")
except ImportError:
    print("Using simplified healthcare model")

# Available healthcare policy models
policy_models = [
    'current_us',      # Status quo
    'usgha',           # US Global Health Assurance
    'm4a',             # Medicare for All
    'uk_nhs',          # UK National Health Service style
    'canadian',        # Canadian single-payer
    'australian',      # Australian hybrid
    'german',          # German multi-payer
    'un_framework'     # UN universal coverage
]

print(f"\nAvailable policy models: {len(policy_models)}")
for i, model in enumerate(policy_models, 1):
    print(f"  {i}. {model}")

In [ ]:
# Exercise 2.1b: Policy comparison framework

# Key metrics for comparison
policy_metrics = {
    'current_us': {
        'coverage_pct': 92,
        'federal_cost_pct_gdp': 5.8,
        'total_cost_pct_gdp': 17.3,
        'admin_overhead_pct': 15,
        'implementation_years': 0
    },
    'usgha': {
        'coverage_pct': 100,
        'federal_cost_pct_gdp': 8.5,
        'total_cost_pct_gdp': 12.0,
        'admin_overhead_pct': 4,
        'implementation_years': 5
    },
    'm4a': {
        'coverage_pct': 100,
        'federal_cost_pct_gdp': 10.5,
        'total_cost_pct_gdp': 13.0,
        'admin_overhead_pct': 3,
        'implementation_years': 4
    },
    'uk_nhs': {
        'coverage_pct': 100,
        'federal_cost_pct_gdp': 9.0,
        'total_cost_pct_gdp': 11.0,
        'admin_overhead_pct': 2,
        'implementation_years': 8
    }
}

# Create comparison table
df = pd.DataFrame(policy_metrics).T
df.index.name = 'Policy'
print("\nHealthcare Policy Comparison:")
print(df.to_string())

In [ ]:
# Exercise 2.1c: Visualize policy trade-offs
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

policies = list(policy_metrics.keys())
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# Coverage vs Federal Cost
ax1 = axes[0]
for i, policy in enumerate(policies):
    m = policy_metrics[policy]
    ax1.scatter(m['federal_cost_pct_gdp'], m['coverage_pct'], 
                s=200, c=colors[i], label=policy, alpha=0.7)
ax1.set_xlabel('Federal Cost (% GDP)')
ax1.set_ylabel('Coverage (%)')
ax1.set_title('Coverage vs Federal Cost')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Total Cost comparison
ax2 = axes[1]
ax2.barh(policies, [policy_metrics[p]['total_cost_pct_gdp'] for p in policies], color=colors)
ax2.set_xlabel('Total Healthcare Cost (% GDP)')
ax2.set_title('Total System Cost')
ax2.grid(True, alpha=0.3, axis='x')

# Administrative Overhead
ax3 = axes[2]
ax3.barh(policies, [policy_metrics[p]['admin_overhead_pct'] for p in policies], color=colors)
ax3.set_xlabel('Administrative Overhead (%)')
ax3.set_title('Admin Efficiency')
ax3.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

### Exercise 2.2: Run Healthcare Monte Carlo Simulation

In [ ]:
# Exercise 2.2a: Monte Carlo simulation for healthcare uncertainty
from core.simulation import Simulation

# Run simulations for two policies
results_current = []
results_usgha = []

# Simplified Monte Carlo
np.random.seed(42)
n_iterations = 1000

for i in range(n_iterations):
    # Current US: High variance
    inflation_shock = np.random.normal(0, 0.015)  # Healthcare inflation
    current_cost = 17.3 * (1 + inflation_shock)
    results_current.append(current_cost)
    
    # USGHA: Lower variance due to cost controls
    usgha_shock = np.random.normal(0, 0.008)
    usgha_cost = 12.0 * (1 + usgha_shock)
    results_usgha.append(usgha_cost)

print(f"Monte Carlo Results ({n_iterations} iterations):")
print(f"\nCurrent US:")
print(f"  Mean: {np.mean(results_current):.2f}% GDP")
print(f"  Std Dev: {np.std(results_current):.3f}%")
print(f"  90% CI: [{np.percentile(results_current, 5):.2f}, {np.percentile(results_current, 95):.2f}]")

print(f"\nUSGHA:")
print(f"  Mean: {np.mean(results_usgha):.2f}% GDP")
print(f"  Std Dev: {np.std(results_usgha):.3f}%")
print(f"  90% CI: [{np.percentile(results_usgha, 5):.2f}, {np.percentile(results_usgha, 95):.2f}]")

In [ ]:
# Exercise 2.2b: Visualize Monte Carlo distributions
fig, ax = plt.subplots(figsize=(12, 6))

ax.hist(results_current, bins=50, alpha=0.6, label='Current US', color='coral', density=True)
ax.hist(results_usgha, bins=50, alpha=0.6, label='USGHA', color='steelblue', density=True)

ax.axvline(np.mean(results_current), color='red', linestyle='--', linewidth=2)
ax.axvline(np.mean(results_usgha), color='blue', linestyle='--', linewidth=2)

ax.set_xlabel('Healthcare Cost (% GDP)')
ax.set_ylabel('Probability Density')
ax.set_title('Monte Carlo Distribution: Healthcare Cost Comparison')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 📝 Analysis Questions - Part 2

1. Which policy offers the best coverage for the lowest federal cost?
2. Why does USGHA have lower total system cost but higher federal cost?
3. What does the narrower distribution for USGHA tell us about risk?
4. What are the trade-offs between implementation time and savings?

---

## Part 3: Social Security Trust Fund Analysis (45 min)

### Exercise 3.1: Trust Fund Mechanics

In [ ]:
# Exercise 3.1a: Model Social Security trust fund
try:
    from core.social_security import SocialSecurityEngine
    print("✅ Social Security module loaded")
except ImportError:
    print("Using simplified Social Security model")

# Trust fund starting values (OASDI combined, billions)
trust_fund_2024 = 2830  # Approximate

# Annual flows
payroll_tax_income = 1214  # Billions
interest_income = 66       # Billions
benefit_payments = 1461    # Billions

annual_flow = payroll_tax_income + interest_income - benefit_payments

print(f"Social Security 2024 Status:")
print(f"  Trust Fund Balance: ${trust_fund_2024:,}B")
print(f"  Annual Income: ${payroll_tax_income + interest_income:,}B")
print(f"  Annual Outgo: ${benefit_payments:,}B")
print(f"  Net Annual Flow: ${annual_flow:,}B {'surplus' if annual_flow > 0 else 'deficit'}")

In [ ]:
# Exercise 3.1b: Project trust fund depletion
years = list(range(2024, 2041))
trust_fund = [trust_fund_2024]

# Growth assumptions
income_growth = 0.035  # 3.5% annual payroll growth
benefit_growth = 0.055  # 5.5% benefit growth (aging + COLA)

current_income = payroll_tax_income + interest_income
current_benefits = benefit_payments

for year in years[1:]:
    current_income *= (1 + income_growth)
    current_benefits *= (1 + benefit_growth)
    
    # Interest depends on trust fund balance
    interest = trust_fund[-1] * 0.023 if trust_fund[-1] > 0 else 0
    
    new_balance = trust_fund[-1] + current_income + interest - current_benefits
    trust_fund.append(max(0, new_balance))

# Find depletion year
depletion_year = None
for i, balance in enumerate(trust_fund):
    if balance == 0:
        depletion_year = years[i]
        break

print(f"\nTrust Fund Projection:")
for i, year in enumerate(years):
    if i % 2 == 0:  # Every other year
        print(f"  {year}: ${trust_fund[i]:,.0f}B")

if depletion_year:
    print(f"\n⚠️ Trust Fund Depletion Year: {depletion_year}")
else:
    print(f"\n✅ Trust fund projected to remain solvent through {years[-1]}")

In [ ]:
# Exercise 3.1c: Visualize trust fund trajectory
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(years, trust_fund, 'b-', linewidth=2, marker='o', markersize=4)
ax.axhline(y=0, color='r', linestyle='--', label='Depletion Level')
ax.fill_between(years, trust_fund, alpha=0.3)

if depletion_year:
    ax.axvline(x=depletion_year, color='red', linestyle=':', alpha=0.8)
    ax.annotate(f'Depletion: {depletion_year}', 
                xy=(depletion_year, 0), 
                xytext=(depletion_year-2, 500),
                arrowprops=dict(arrowstyle='->', color='red'),
                fontsize=12, color='red')

ax.set_xlabel('Year')
ax.set_ylabel('Trust Fund Balance ($ Billions)')
ax.set_title('Social Security Trust Fund Projection')
ax.grid(True, alpha=0.3)
ax.legend()

plt.tight_layout()
plt.show()

### Exercise 3.2: Reform Scenarios

In [ ]:
# Exercise 3.2: Model reform scenarios

def project_trust_fund(years, starting_balance, base_income, base_benefits,
                       income_growth=0.035, benefit_growth=0.055,
                       payroll_tax_increase=0, benefit_reduction=0):
    """Project trust fund with optional reforms."""
    trust_fund = [starting_balance]
    income = base_income * (1 + payroll_tax_increase)
    benefits = base_benefits * (1 - benefit_reduction)
    
    for year in years[1:]:
        income *= (1 + income_growth)
        benefits *= (1 + benefit_growth)
        interest = trust_fund[-1] * 0.023 if trust_fund[-1] > 0 else 0
        new_balance = trust_fund[-1] + income + interest - benefits
        trust_fund.append(max(0, new_balance))
    
    return trust_fund

# Scenarios
scenarios = {
    'Baseline': project_trust_fund(years, trust_fund_2024, 1280, 1461),
    '+2% Payroll Tax': project_trust_fund(years, trust_fund_2024, 1280, 1461, 
                                          payroll_tax_increase=0.16),  # 12.4% → 14.4%
    '-15% Benefits': project_trust_fund(years, trust_fund_2024, 1280, 1461,
                                        benefit_reduction=0.15),
    'Combined Reform': project_trust_fund(years, trust_fund_2024, 1280, 1461,
                                          payroll_tax_increase=0.08,
                                          benefit_reduction=0.08)
}

# Plot all scenarios
plt.figure(figsize=(14, 7))
colors = ['#1f77b4', '#2ca02c', '#ff7f0e', '#9467bd']
for (name, values), color in zip(scenarios.items(), colors):
    plt.plot(years, values, label=name, linewidth=2, color=color)

plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.xlabel('Year')
plt.ylabel('Trust Fund Balance ($ Billions)')
plt.title('Social Security Reform Scenarios')
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 📝 Analysis Questions - Part 3

1. Which reform scenario keeps the trust fund solvent longest?
2. What are the political trade-offs between raising taxes vs cutting benefits?
3. Why does the combined reform require smaller changes in each area?
4. How do these results compare to the SSA Trustees Report?

---

## Part 4: Synthesis Exercise (30 min)

### Exercise 4: Design Your Own Policy Package

In [ ]:
# Exercise 4: Create a comprehensive fiscal policy package

# Your policy parameters (MODIFY THESE)
my_policy = {
    # Healthcare
    'healthcare_model': 'current_us',  # Options: current_us, usgha, m4a, uk_nhs
    
    # Social Security
    'payroll_tax_change': 0.0,   # % increase to payroll tax rate
    'benefit_reduction': 0.0,     # % reduction in benefits
    
    # Discretionary
    'defense_change': 0.0,        # % change to defense spending
    'nondefense_change': 0.0,     # % change to non-defense discretionary
    
    # Revenue
    'income_tax_change': 0.0,     # % point change to top rate
    'corporate_tax_change': 0.0,  # % point change to corporate rate
}

# Evaluate your policy
print("Your Policy Package:")
print("="*50)
for key, value in my_policy.items():
    print(f"  {key}: {value}")

print("\n💡 Modify the values above and re-run to experiment!")

In [ ]:
# Exercise 4b: Evaluate policy impact

def evaluate_policy(policy):
    """Estimate 10-year fiscal impact of policy package."""
    baseline_deficit_2034 = 2.5  # trillion
    
    # Healthcare impact
    hc_savings = {
        'current_us': 0,
        'usgha': 0.5,  # Lower total costs but higher federal
        'm4a': -0.3,   # Higher federal costs
        'uk_nhs': 0.4
    }
    hc_impact = hc_savings.get(policy['healthcare_model'], 0)
    
    # Social Security impact
    ss_revenue = policy['payroll_tax_change'] * 15  # $15B per 1% tax
    ss_savings = policy['benefit_reduction'] * 15    # $15B per 1% cut
    
    # Discretionary impact
    defense_impact = policy['defense_change'] * 0.009  # $9B per 1%
    nondefense_impact = policy['nondefense_change'] * 0.007
    
    # Revenue impact
    income_tax_impact = policy['income_tax_change'] * 50  # $50B per point
    corporate_impact = policy['corporate_tax_change'] * 10
    
    total_impact = (hc_impact + ss_revenue/1000 + ss_savings/1000 - 
                   defense_impact - nondefense_impact + 
                   income_tax_impact/1000 + corporate_impact/1000)
    
    new_deficit = baseline_deficit_2034 - total_impact
    
    return {
        'baseline_deficit': baseline_deficit_2034,
        'policy_impact': total_impact,
        'new_deficit': new_deficit,
        'improvement_pct': (total_impact / baseline_deficit_2034) * 100
    }

results = evaluate_policy(my_policy)

print("\nPolicy Impact Assessment (2034):")
print("="*50)
print(f"  Baseline Deficit: ${results['baseline_deficit']:.2f}T")
print(f"  Policy Impact: ${results['policy_impact']:.2f}T {'savings' if results['policy_impact'] > 0 else 'cost'}")
print(f"  New Deficit: ${results['new_deficit']:.2f}T")
print(f"  Improvement: {results['improvement_pct']:.1f}%")

## 🎯 Workshop Completion

**Congratulations!** You've completed the half-day policy workshop.

### Skills Acquired:
- ✅ Revenue and spending decomposition
- ✅ Healthcare policy comparison
- ✅ Monte Carlo uncertainty analysis
- ✅ Social Security trust fund modeling
- ✅ Policy package design and evaluation

### Next Steps:
1. Explore the full-day technical workshop for advanced modeling
2. Run your policy through the PoliSim dashboard
3. Review the detailed notebooks for deeper analysis

---

*Questions? Visit [GitHub](https://github.com/GalacticOrgOfDev/polisim)*